# Learn Posture

use machine learning to recognize robot's posture (following the example in [scikit-learn-intro.ipynb](./scikit-learn-intro.ipynb) )

## 1. Data collection

We have colleceted data before, you need to add new data if you want to add new posture.

* the dateset are in *robot_pose_data* folder
* each file contains the data belongs to this posture, e.g. the data in *Back* file are collected when robot was in "Back" posture
* the data file can be load by ```pickle```, e.g. ```pickle.load(open('Back'))```, the data is a list of feature data
* the features (e.g. each row of the data) are ['LHipYawPitch', 'LHipRoll', 'LHipPitch', 'LKneePitch', 'RHipYawPitch', 'RHipRoll', 'RHipPitch', 'RKneePitch', 'AngleX', 'AngleY'], where 'AngleX' and 'AngleY' are body angle (e.g. ```Perception.imu```) and others are joint angles.

## 2. Data preprocessing

In [9]:
%pylab inline
import pickle
from os import listdir, path
import numpy as np
from sklearn import svm, metrics

ROBOT_POSE_DATA_DIR = 'robot_pose_data'

Populating the interactive namespace from numpy and matplotlib


In [10]:
classes = listdir(ROBOT_POSE_DATA_DIR)
print classes

['Frog', 'Sit', 'Back', 'Knee', 'Crouch', 'HeadBack', 'StandInit', 'Right', 'Stand', 'Left', 'Belly']


In [24]:
def load_pose_data(i):
    '''load pose data from file'''
    data = []
    target = []
    # YOUR CODE HERE
    
    filename = path.join(ROBOT_POSE_DATA_DIR, classes[i])
    data = pickle.load(open(filename))
    target = [i] * len(data)
    return data, target

In [14]:
# load all the data
all_data = []
all_target = []
# YOUR CODE HERE
for i in range (len(classes)):
    data, target = load_pose_data(i)
    all_data = all_data + data
    all_target = all_target + target 
    
print 'total number of data', len(all_data)

total number of data 222


In [15]:
# shuffule data
permutation = np.random.permutation(len(all_data))
n_training_data = int(len(all_data) * 0.7)
training_data = permutation[:n_training_data]

## 3. Learn on training data

In scikit-learn, an estimator for classification is a Python object that implements the methods fit(X, y) and predict(T). An example of an estimator is the class sklearn.svm.SVC that implements support vector classification.

In [16]:
clf = svm.SVC(gamma=0.001, C=100.)

### learning

In [17]:
# YOUR CODE HERE
clf.fit(all_data,all_target)

SVC(C=100.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma=0.001, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

### predicting

In [18]:
clf.predict(all_data[-1]), all_target[-1]

/usr/local/lib/python2.7/dist-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


(array([10]), 10)

In [19]:
def evaluate(expected, predicted):
    print("Classification report:\n%s\n" % metrics.classification_report(expected, predicted))

    print("Confusion matrix:\n%s" % metrics.confusion_matrix(expected, predicted))

In [20]:
expected = []
predicted = []
# YOUR CODE HERE

predicted = clf.predict(all_data)
expected = all_target

evaluate(expected, predicted)

Classification report:
             precision    recall  f1-score   support

          0       1.00      1.00      1.00        10
          1       1.00      1.00      1.00        26
          2       1.00      1.00      1.00        23
          3       1.00      1.00      1.00        10
          4       0.97      1.00      0.98        30
          5       1.00      1.00      1.00        10
          6       1.00      1.00      1.00        52
          7       1.00      0.91      0.95        11
          8       1.00      1.00      1.00        11
          9       1.00      1.00      1.00        20
         10       1.00      1.00      1.00        19

avg / total       1.00      1.00      1.00       222


Confusion matrix:
[[10  0  0  0  0  0  0  0  0  0  0]
 [ 0 26  0  0  0  0  0  0  0  0  0]
 [ 0  0 23  0  0  0  0  0  0  0  0]
 [ 0  0  0 10  0  0  0  0  0  0  0]
 [ 0  0  0  0 30  0  0  0  0  0  0]
 [ 0  0  0  0  0 10  0  0  0  0  0]
 [ 0  0  0  0  0  0 52  0  0  0  0]
 [ 0  0  0  0 

## 4. Evaluate on the test data

In [21]:
expected = []
predicted = []
# YOUR CODE HERE
predicted = clf.predict(all_data)
expected = all_target


evaluate(expected, predicted)

Classification report:
             precision    recall  f1-score   support

          0       1.00      1.00      1.00        10
          1       1.00      1.00      1.00        26
          2       1.00      1.00      1.00        23
          3       1.00      1.00      1.00        10
          4       0.97      1.00      0.98        30
          5       1.00      1.00      1.00        10
          6       1.00      1.00      1.00        52
          7       1.00      0.91      0.95        11
          8       1.00      1.00      1.00        11
          9       1.00      1.00      1.00        20
         10       1.00      1.00      1.00        19

avg / total       1.00      1.00      1.00       222


Confusion matrix:
[[10  0  0  0  0  0  0  0  0  0  0]
 [ 0 26  0  0  0  0  0  0  0  0  0]
 [ 0  0 23  0  0  0  0  0  0  0  0]
 [ 0  0  0 10  0  0  0  0  0  0  0]
 [ 0  0  0  0 30  0  0  0  0  0  0]
 [ 0  0  0  0  0 10  0  0  0  0  0]
 [ 0  0  0  0  0  0 52  0  0  0  0]
 [ 0  0  0  0 

## 5. Deploy to the real system

We can simple use `pickle` module to serialize the trained classifier.

In [22]:
import pickle
ROBOT_POSE_CLF = 'robot_pose.pkl'
pickle.dump(clf, open(ROBOT_POSE_CLF, 'w'))

Then, in the application we can load the trained classifier again.

In [23]:
clf2 = pickle.load(open(ROBOT_POSE_CLF))
clf2.predict(all_data[-1]), all_target[-1]

/usr/local/lib/python2.7/dist-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


(array([10]), 10)